In [1]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("Random Forest Classification Example");
    var sc = new SparkContext(sparkConf);
    var MLUtils = require("eclairjs/mllib/MLUtils");
    var RandomForest = require('eclairjs/mllib/tree/RandomForest');
    var Tuple = require('eclairjs/Tuple');

var datapath =  "../../data/mllib/sample_libsvm_data.txt";


In [2]:
    var data = MLUtils.loadLibSVMFile(sc, datapath);


 Split the data into training and test sets (30% held out for testing)


In [3]:
    var splits = data.randomSplit([0.7, 0.3]);
    var trainingData = splits[0];
    var testData = splits[1];


 Train a RandomForest model.
 Empty categoricalFeaturesInfo indicates all features are continuous.


In [4]:
    var numClasses = 2;
    var categoricalFeaturesInfo = {};
    var numTrees = 3; // Use more in practice.
    var featureSubsetStrategy = "auto"; // Let the algorithm choose.
    var impurity = "gini";
    var maxDepth = 5;
    var maxBins = 32;
    var seed = 12345;


 Train a RandomForest model.


In [5]:
    var model = RandomForest.trainClassifier(
        trainingData,
        numClasses,
        categoricalFeaturesInfo,
        numTrees,
        featureSubsetStrategy,
        impurity,
        maxDepth,
        maxBins,
        seed
    );


 Evaluate model on test instances and compute test error


In [6]:
    var predictionAndLabel = testData.mapToPair(function (p, model, Tuple) {
        return new Tuple(model.predict(p.getFeatures()), p.getLabel());
    }, [model, Tuple]);

    var testErr = 1.0 * predictionAndLabel.filter(function (tup) {
            return (tup[0] !== tup[1]);
        }).count() / testData.count();
    var ret = {};
    ret.testErr = testErr;
    ret.model = model;
var result = ret;
    print("Test Error: " + result.testErr);
    print("Learned classification forest model:\n" + result.model.toDebugString());


 Save and load model


In [7]:
    result.model.save(sc, "target/tmp/myRandomForestClassificationModel");
    var RandomForestModel = require('eclairjs/mllib/tree/model/RandomForestModel');
    var sameModel = RandomForestModel.load(
        sc,
        "target/tmp/myRandomForestClassificationModel"
    );

    sc.stop();
